In [25]:
import numpy as np

np.random.seed(123)
X = np.load('../generated_games/features-40k.npy')
Y = np.load('../generated_games/labels-40k.npy')
print("features", X.shape)
print("labels", Y.shape)

samples = X.shape[0]
size = 9
input_shape = (size, size, 1)

X = X.reshape(samples, size, size, 1)

train_samples = int(0.9 * samples)
X_train, X_test = X[:train_samples], X[train_samples:]
Y_train, Y_test = Y[:train_samples], Y[train_samples:]
print("X_train", X_train.shape)
print("Y_train", Y_train.shape)
print("X_test", X_test.shape)
print("Y_test", Y_test.shape)

features (41439, 1, 9, 9)
labels (41439, 81)
X_train (37295, 9, 9, 1)
Y_train (37295, 81)
X_test (4144, 9, 9, 1)
Y_test (4144, 81)


In [46]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input

model = Sequential()
model.add(Input(shape=input_shape))
model.add(Conv2D(filters=48,
                 kernel_size=(3, 3),
                 padding='same',
                 activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Conv2D(48, (3, 3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.5))
model.add(Flatten()) 
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(size * size, activation='softmax')) 
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_13 (Conv2D)              │ (None, 9, 9, 48)       │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 9, 9, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 9, 9, 48)       │        20,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 4, 4, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 4, 4, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 512)            │       393,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 81)             │        41,553 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 456,545 (1.74 MB)

 Trainable params: 456,545 (1.74 MB)

 Non-trainable params: 0 (0.00 B)

In [47]:
import time

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# Record the start time
start_time = time.time()
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=200,
          verbose=1,
          validation_data=(X_test, Y_test))
# Record the end time
end_time = time.time()
# Calculate and display the wall time
print(f"Wall time: {end_time - start_time:.6f} seconds")

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.0153 - loss: 4.3937 - val_accuracy: 0.0224 - val_loss: 4.3135
Epoch 2/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.0217 - loss: 4.3144 - val_accuracy: 0.0224 - val_loss: 4.2717
Epoch 3/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.0233 - loss: 4.2821 - val_accuracy: 0.0261 - val_loss: 4.2527
Epoch 4/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.0239 - loss: 4.2649 - val_accuracy: 0.0253 - val_loss: 4.2397
Epoch 5/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.0238 - loss: 4.2522 - val_accuracy: 0.0251 - val_loss: 4.2212
Epoch 6/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.0233 - loss: 4.2320 - val_accuracy: 0.0241 - val_loss: 4.2002
Epoch 7/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.0240 - loss: 4.2055 - val_accuracy: 0.0268 - val_loss: 4.1739
Epoch 8/200
583/583 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.0261 - loss: 4.1771 - 

In [51]:
test_board = np.array([[
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 0,  1, -1,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
]]).reshape(1, 9, 9, 1)
move_probs = model.predict(test_board)[0]
i = 0
for row in range(9):
    row_formatted = []
    for col in range(9):
        row_formatted.append('{:.3f}'.format(move_probs[i]))
        i += 1
    print(' '.join(row_formatted))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
0.000 0.001 0.001 0.001 0.001 0.001 0.000 0.001 0.000
0.001 0.006 0.013 0.032 0.022 0.008 0.004 0.002 0.001
0.002 0.016 0.003 0.093 0.048 0.039 0.001 0.003 0.001
0.002 0.013 0.021 0.025 0.021 0.014 0.008 0.004 0.001
0.003 0.015 0.009 0.006 0.021 0.025 0.012 0.006 0.001
0.002 0.021 0.028 0.033 0.095 0.065 0.020 0.006 0.001
0.001 0.010 0.002 0.057 0.054 0.030 0.001 0.005 0.001
0.000 0.005 0.008 0.016 0.011 0.007 0.007 0.003 0.000
0.000 0.000 0.000 0.001 0.001 0.000 0.000 0.000 0.000


Starlight MacBook Air M2:
> Wall time: 1366.246495 seconds

> Test loss: 3.5951743125915527
> Test accuracy: 0.13778957724571228

MacBookPro M1 Pro:
> Wall time: 